## Equilibrium and stability basic analysis
### (Canard sizing)

Nemo human powered hydrofoil

August 2016 - Gustavo Violato, Diego Montero, Fernando Valentini

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import LinearAxis, Range1d, Label
from bokeh.io import push_notebook
import ipywidgets as wd
import basic_equil as beq
import readouts as ro
import yaml
%matplotlib notebook

In [10]:
output_notebook()

Loading BokehJS ...

In [4]:
# Global Constants
D2R = np.pi/180.
R2D = 1./D2R

DIMSFILE = 'nemo_dims.yml'

In [5]:
# Flight parameters
fp = {'V':4.,          # Just a dummy value, it will be replaced as needed
      'iw':2.5*D2R,    # Wind incidence angle (radians)
      'delta_cg':0.05, # CG displacement wrt nominal position
      'mass': 101      # Total mass
     }

In [6]:
# Nemo relevant equilibrim dimensions/characteristics
dims = beq.read_dims(DIMSFILE)
# Speeds for analysis:
# Takeoff speeds
Vs_TO = np.arange(1.,3.1,0.25)
# Flight speeds
Vs = np.arange(3.3,7.0,0.3)
# Exepcted Thrust (to be checked against drag curve later)
Ts = np.array([66.81,62.09,60.08,60.00,61.33,63.79,67.29,71.84,77.63,84.65,95.37,112.26,140.75])
# Equilibrium Matrix
eq_matrix = beq.solve_equilibrium(Vs,dims,fp,Thrust=Ts)
# AVL Calculaleted Equilibrium Matrix
#avl_eq_TO = ro.read_results('./results_to',Vs_TO)
avl_eq, avl_st = ro.read_results('./results_7mps',Vs)
#avl_eq = np.append(avl_eq_TO,avl_eq_FL,axis=0)

In [7]:
#Plotting
p = figure(title='Equilibrium conditions',
           x_axis_label='Flight Speed [m/s]',
           y_axis_label='AoA and d_p [deg]',
           y_range=[-5,10],
           width=900, height=350, toolbar_location='above')

aoa = p.line(eq_matrix[:,0],eq_matrix[:,1], line_color='navy',legend='AoA')
dp  = p.line(eq_matrix[:,0],eq_matrix[:,2], line_color='firebrick',legend='dp')

p.line(avl_eq[:,0],avl_eq[:,1], line_color='blue',legend='AVL_AoA')
p.line(avl_eq[:,0],avl_eq[:,2], line_color='red',legend='AVL_dp')

p.extra_y_ranges = {"cls": Range1d(start=0., end=1.2)}
p.add_layout(LinearAxis(y_range_name="cls"), 'right')

clw = p.line(eq_matrix[:,0], eq_matrix[:,4], line_color="green", y_range_name="cls", legend='Cl_w')
clc = p.line(eq_matrix[:,0], eq_matrix[:,6], line_color="teal", y_range_name="cls", legend='Cl_c')

p.line(avl_eq[:,0], avl_eq[:,4], line_color="darkgreen", y_range_name="cls", legend='AVL_Cl_w')
p.line(avl_eq[:,0], avl_eq[:,5], line_color="cadetblue", y_range_name="cls", legend='AVL_Cl_c')

In [8]:
def update(iw=2.5, d_cg=0.05, mass=101.):
    fp['iw'] = iw*D2R
    fp['delta_cg'] = d_cg
    fp['mass'] = mass
    eq_matrix = beq.solve_equilibrium(Vs,dims,fp)
    aoa.data_source.data['y'] = eq_matrix[:,1]
    dp.data_source.data['y']  = eq_matrix[:,2]
    clw.data_source.data['y'] = eq_matrix[:,4]
    clc.data_source.data['y'] = eq_matrix[:,6]
    push_notebook()

In [11]:
show(p)

In [12]:
wd.interact(update,iw=(-2,6,0.1),d_cg=(-0.1,0.2,0.01),mass=(90.,110.,0.5))

<function __main__.update>

In [13]:
p99 = figure(title='Equilibrium conditions',
           x_axis_label='AoA [deg]',
           y_axis_label='Cd',
           width=900, height=350, toolbar_location='above')

p99.line(avl_eq[:,1],avl_eq[:,6], line_color='blue',legend='Cd_tot')
p99.line(avl_eq[:,1],avl_eq[:,8], line_color='red',legend='Cd_ind')

In [14]:
show(p99)

In [15]:
p98 = figure(title='Equilibrium conditions',
           x_axis_label='AoA [deg]',
           y_axis_label='Cd',
           width=900, height=350, toolbar_location='above')


#p99.line(avl_eq[:,1],avl_eq[:,6], line_color='blue',legend='Cd_tot')
#p99.line(avl_eq[:,1],avl_eq[:,8], line_color='red',legend='Cd_ind')
p98.line(avl_eq[:,1],avl_eq[:,6]-avl_eq[:,8], line_color='green')

In [16]:
show(p98)

In [17]:
print 'w_root_c: ', W_ROOT_C
print 'c_root_c: ', C_ROOT_C
print 'cref: ', W_CMA
print 'w_root_t: ', W_ROOT_T
print 'strut_t: ', STRUT_T
print 'strut_x: ', STRUT_X
print 'ws_drag_factor: ', WS_DRAG_FACTOR
print 'rudder_t: ', RUDDER_T
print 'canard_t: ', CANARD_T
print 'w_AR: ', W_A
print 's_hull: ', S_HULL

w_root_c: 

NameError: name 'W_ROOT_C' is not defined

In [18]:
# Drag build-up!

# Method: Calculate Cd contributions for each speed always normalizing on Sref (wing area).
#         Then sum them up and multiply by q(V)*Sref for each speed

# Important dimensions/variables
wing = dims['wing']
cnrd = dims['canard']

GRAV = 9.80665 # Gravity
RHO_W = 1020   # Sea water density
RHO_AIR = 1.225 # Air density

W_ROOT_C = np.sqrt(wing['S']/wing['AR'])*2/(1+wing['lambda']) # Wing Root Chord
C_ROOT_C = np.sqrt(cnrd['S']/cnrd['AR'])*2/(1+cnrd['lambda']) # Canard Root Chord
W_CMA    = beq.cma_s(wing) # Wing Mean Aerod. Chord

W_ROOT_T = 0.12*W_ROOT_C # Wing Root Thickness
STRUT_T  = 0.03          # Main Strut Thickness
STRUT_X  = 0.3*0.187     # Distance along strut chord with max thickness
WS_DRAG_FACTOR = 0.5     # Factor to account for wing/main strut staggering

RUDDER_T = 0.15*0.06     # Rudder Thickness
CANARD_T = 0.11*C_ROOT_C # Canard Thickness

W_A    = 17     # Wing aspect ratio
S_REF  = 0.191  # Reference area (Wing Area)
S_HULL = 0.054 # Hull cross sectional area

# Total Hidrodynamic drag from AVL (based on Sref)
Vs = avl_eq[:,0]
Cd_avl = avl_eq[:,6]

# Increasing induced drag from wing about 10% as Hoerner says... (XI-C, ~11-26)
# "A single strut in the center increases drag due to lift by some 10%"
Cdi_inc = 0.1*avl_eq[:,7]

# Hydrofoil wave drag, based on Breslin, J. "Hydrofoil Wave Drag Theory"
# Wing depth is supposed to be around a quarter of span ~1.8/4=0.45m
F_c    = Vs/np.sqrt(GRAV*W_CMA)
Cd_h_w = avl_eq[:,4]**2*(0.5/(np.pi*W_A) + 0.13/F_c**2)

# Interference drag, based on Hoerner VIII-4 (8-10)
# Cdt = Drag/(qt^2) = 17*(t/c)^2-0.05
ws_t = (W_ROOT_T + STRUT_T)/2
rc_t = (RUDDER_T + CANARD_T)/2

Cd_int_ws = (17*(ws_t/W_ROOT_C)**2-0.05)*ws_t**2/S_REF*WS_DRAG_FACTOR*np.ones(Cd_avl.shape)
Cd_int_rc = (17*(rc_t/C_ROOT_C)**2-0.05)*rc_t**2/S_REF*np.ones(Cd_avl.shape)

# Spray drag, based on Hoerner X-C (10-13)
# Cdx = Drag/(qx^2) = 0.24*(t/x)^2
Cd_spray = 0.24*(STRUT_T/STRUT_X)**2*STRUT_X**2/S_REF*np.ones(Cd_avl.shape)

# Air resistance (Rider + Hulls), based on Hoerner (3-13), Rider and
# Hoerner XIII-1 (13-1) Hulls
Cd_air = (0.54 + 0.1*2*S_HULL)/S_REF*np.ones(Cd_avl.shape)

# Build-up
Cd_total = Cd_avl + Cdi_inc + Cd_h_w + Cd_int_ws + Cd_int_rc + Cd_spray

# Arbitrary safety factor for other drag sources such as:
# - Air resistance of frame and interferences
# - Drag from surface follower
# - Spray drag from canard
# - Propeller/Strut interaction
Cd_total *= 1.1

# ...aaaaand we're done!
Drag_T = np.multiply(Cd_total,Vs**2)*0.5*RHO_W*S_REF
Drag_T += np.multiply(Cd_air,Vs**2)*0.5*RHO_AIR*S_REF

#mlt_data = np.loadtxt('drag.mlt')
#Hull_Drag = mlt_data[:,1] + mlt_data[:,2]
#Drag_T[:len(Hull_Drag)-1] += Hull_Drag[:-1]*1000

In [19]:
for i,D in enumerate(Drag_T):
    print avl_eq[i,0],D

3.3 64.5179641675
3.6 60.0162155513
3.9 58.2036473365
4.2 58.2964085606
4.5 59.7698400442
4.8 62.3661785668
5.1 65.9789186068
5.4 70.6173110321
5.7 76.4891388988
6.0 83.5841169216
6.3 94.3549794593
6.6 111.269788547
6.9 139.754996509


In [23]:
3.6*3.8*0.5*0.7

4.787999999999999

In [21]:
p3 = figure(title='Drag and Available Thrust',
            x_axis_label='Flight Speed [m/s]',
            x_range=[1,7],
            y_axis_label='Drag [N]',
            y_range=[0,150],
            width=900, height=350)

p3.line(avl_eq[:,0],Drag_T, line_color='black',legend='Drag')

PWR_LVL = [200,250,300,350,500,700] # Power Level in Watts
Vs = np.linspace(1,7)
for i,pl in enumerate(PWR_LVL):
    Thrust = np.divide(pl*np.ones(Vs.shape),Vs)
    p3.line(Vs,Thrust, line_color='blue', line_dash='dashed')
    p3.add_layout(Label(x=pl/140.-0.21-i/5*0.13, y=140, text_font_size='10pt',
                        angle=-np.pi/3.5+i*np.pi/40, text='{}W'.format(pl)))
    
    
V_prop = np.arange(0.5,6.1,0.5)
T_prop_max = np.array([94,97,103,113,125,135,135,131,125,117,109,88])
T_prop_250 = np.array([51,56,62,67,68,65,69,53,48,43,38,33])

p3.line(V_prop,T_prop_max,color='red',legend='Max Thrust')
p3.line(V_prop,T_prop_250,color='green',legend='250W Thrust')

show(p3)

In [ ]:
#Plotting
p2 = figure(title='Equilibrium conditions',
           x_axis_label='Flight Speed [m/s]',
           y_axis_label='AoA and d_p [deg]',
           width=900, height=350)

p2.line(eq_matrix[:,0],eq_matrix[:,-2], line_color='navy',legend='Xn')
p2.line(eq_matrix[:,0],eq_matrix[:,-1], line_color='firebrick',legend='Xcg')

p2.line(avl_eq[:,0],-1*avl_eq[:,-1]+0.25*W_ROOT_C, line_color='blue',legend='AVL_Xn')
p2.line(avl_eq[:,0],-1*avl_eq[:,-2]+0.25*W_ROOT_C, line_color='red',legend='AVL_xcg')

show(p2)